In [7]:
import numpy as np
import matplotlib.pyplot as plt
import boost_histogram as bh
import tensorflow as tf
import sklearn as sk
import pandas as pd
import pyarrow.feather as feather
import sklearn.model_selection as model_selection
import os

class DataLoader:
    def __init__(self, file_name, selected_labels):
        self.file_name = file_name
        self.selected_labels = selected_labels
        self.x_data, self.y_data, self.weight_data = self.import_feather()

    def import_feather(self):
        df = feather.read_feather(self.file_name)
        df = df.set_index(['entry', 'subentry'])

        weight_data = df['Event.Weight'][:,0].values.reshape(-1, 1)
        weight_data = np.vstack([weight_data, weight_data])
        y_data = np.array(weight_data)
        y_data = (y_data > 0).astype(int)

        feature_columns = df.columns.intersection(self.selected_labels).tolist()
        selected_df = df[feature_columns].copy()

        flip_mask = selected_df.columns.str.contains(r'\.PT|\.Eta')
        phi_mask = selected_df.columns.str.contains(r'\.Phi')
        selected_df.loc[:, flip_mask] *= -1 
        selected_df.loc[:, phi_mask] = (selected_df.loc[:, phi_mask] + np.pi) % (2 * np.pi) - np.pi

        Pflipped_df = (selected_df.assign(counter=selected_df.groupby('entry').cumcount())
                       .pivot_table(index='entry', columns=['subentry'], values=feature_columns, dropna=False, fill_value=0))

        selected_df = (selected_df.assign(counter=selected_df.groupby('entry').cumcount())
                       .pivot_table(index='entry', columns=['subentry'], values=feature_columns, dropna=False, fill_value=0))
        x_data = np.concatenate([selected_df.to_numpy(), Pflipped_df.to_numpy()], axis=0)

        print('x_data, y_data, weight_data', x_data.shape, y_data.shape, weight_data.shape)
        return x_data, y_data, weight_data

class CustomKerasClassifier(sk.base.BaseEstimator, sk.base.ClassifierMixin):
    fit_count = 0 
    total_fits = 1

    def __init__(self, input_shape, learning_rate=0.001, neurons=16, layers=2, reg_strength=0.001, epochs=50, patience=2, dropout_rate=0, batch_size=128):
        self.input_shape = input_shape
        self.learning_rate = learning_rate
        self.neurons = neurons
        self.layers = layers
        self.reg_strength = reg_strength
        self.epochs = epochs
        self.patience = patience
        self.batch_size = batch_size
        self.dropout_rate = dropout_rate
        self.model = None

    def build_model(self):
        model = tf.keras.models.Sequential()
        model.add(tf.keras.Input(shape=(self.input_shape,)))

        for _ in range(self.layers-1):
            model.add(tf.keras.layers.Dense(self.neurons, activation='relu'))

        model.add(tf.keras.layers.Dense(self.neurons, kernel_regularizer=tf.keras.regularizers.l2(self.reg_strength)))
        model.add(tf.keras.layers.Dropout(rate=self.dropout_rate))
        model.add(tf.keras.layers.Dense(2, activation='softmax'))
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=self.learning_rate),
                      loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        return model

    def fit(self, x_train, y_train, validation_data=(None, None)):
        CustomKerasClassifier.fit_count += 1
        print("\n" f"Running fit {CustomKerasClassifier.fit_count}/{CustomKerasClassifier.total_fits}")

        self.model = self.build_model()
        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', patience=self.patience, restore_best_weights=True
        )

        history = self.model.fit(
            x_train, y_train, 
            validation_data=validation_data,  
            epochs=self.epochs, 
            batch_size=self.batch_size,
            verbose=2, 
            callbacks=[early_stopping]
        )

        self.history_ = history.history

        return self

    def predict(self, x):
        return self.model.predict(x).argmax(axis=1)

    def score(self, x, y):
        _, accuracy = self.model.evaluate(x, y, verbose=0)
        return accuracy

class MLModel:
    def __init__(self):
        self.model = None

    def data_preprocessing(self, x_data, y_data):
        x_data = sk.preprocessing.StandardScaler().fit_transform(x_data)
        x_train, x_test, y_train, y_test = model_selection.train_test_split(x_data, y_data, test_size=0.2, random_state=42)
        x_validate, x_test, y_validate, y_test = model_selection.train_test_split(x_test, y_test, test_size=0.5, random_state=42)
        
        return x_train, x_test, y_train, y_test, x_validate, y_validate

    def grid_search(self, x_data, y_data, param_grid, model_filename):
        x_train, x_test, y_train, y_test, x_validate, y_validate = self.data_preprocessing(x_data, y_data)
        input_shape = x_train.shape[1]

        CustomKerasClassifier.total_fits = np.prod([len(v) for v in param_grid.values()]) * 2
        CustomKerasClassifier.fit_count = 0

        model = CustomKerasClassifier(input_shape=input_shape)
        grid = model_selection.GridSearchCV(estimator=model, param_grid=param_grid, cv=2, n_jobs=1, verbose=2)
        grid_result = grid.fit(x_train, y_train, validation_data=(x_validate, y_validate))

        print("Best Accuracy: {:.4f} using {}".format(grid_result.best_score_, grid_result.best_params_))
        self.model = grid_result.best_estimator_.model
        self.model.save(model_filename)
        print(self.model.history.history)
        history_df = pd.DataFrame.from_dict(self.model.history.history, orient="columns")
        print("history_df",history_df)
        feather.write_feather(history_df, model_filename.replace(".keras", "_history.feather"))

    def predict(self, x_data):
        x_data = sk.preprocessing.StandardScaler().fit_transform(x_data)
        prediction = self.model.predict(x_data)
        return prediction
    
    def load_ML_model(model_filename):
        ml_model = MLModel()
        ml_model.model = tf.keras.models.load_model(model_filename)
        history_df = feather.read_feather(model_filename.replace(".keras", "_history.feather"))
        
        return ml_model, history_df
    
class Plotter:
    @staticmethod
    def plot_loss(history):
        training_loss = history['loss'] 
        validation_loss = history['val_loss']
        epochs = np.arange(1, len(training_loss) + 1)
        
        plt.figure(figsize=(8, 5))
        plt.plot(epochs, training_loss, marker='x', linestyle='-', color='mediumblue', label='Training')
        plt.plot(epochs, validation_loss, marker='x', linestyle='-', color='red', label='Validation')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.title('Loss Function Over Epochs')
        plt.legend()
        plt.show() 
 
    @staticmethod
    def plot_accuracy(history):
        training_accuracy = history['accuracy'] 
        validation_accuracy = history['val_accuracy']
        epochs = np.arange(1, len(training_accuracy) + 1)

        plt.figure(figsize=(8, 5))
        plt.plot(epochs, training_accuracy, marker='x', linestyle='-', color='mediumblue', label='Training')
        plt.plot(epochs, validation_accuracy, marker='x', linestyle='-', color='red', label='Validation')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.title('Accuracy Over Epochs')
        plt.legend()
        plt.show()

    @staticmethod
    def chi_squared(observed, expected, uncertainty):
        chi_squared = np.sum(((observed - expected) / uncertainty) ** 2)
        return chi_squared

    @staticmethod        
    def plot_ML_observable(ML_results, weight_data, region, interation):
        scale_factor = 1
        ML_observable = ML_results[:, 1] - ML_results[:, 0]

        bin_number = 20
        range_limit = (-1, 1)

        histogram_0 = bh.Histogram(bh.axis.Regular(bin_number, *range_limit, underflow=False, overflow=False))
        print("ML and Weight shape", ML_observable.shape, weight_data.shape)
        histogram_0.fill(ML_observable, weight=weight_data)

        bin_widths = histogram_0.axes[0].widths
        differential_cross_section_0 = histogram_0.view() / bin_widths

        hist_0_squared = bh.Histogram(histogram_0.axes[0])
        hist_0_squared.fill(ML_observable, weight=weight_data**2)

        uncertainty_0 = np.sqrt(hist_0_squared.view()) / bin_widths

        bin_centers = histogram_0.axes[0].centers
        bin_edges = histogram_0.axes[0].edges

        chi_squared_1 = Plotter.chi_squared(differential_cross_section_0, 0, uncertainty_0)
        print("chi_squared_1 about y=0:", chi_squared_1)

        plt.figure()

        plt.hist(bin_edges[:-1], bins=bin_edges, weights=differential_cross_section_0 /scale_factor, label=r"$O_{NN}$", color='mediumblue', histtype='step')
        plt.errorbar(bin_centers, differential_cross_section_0 / scale_factor, yerr=uncertainty_0 /scale_factor, fmt='None', color='mediumblue')

        plt.title(f'$O_{{NN}}$ for ${region}$ region', fontsize=14)
        plt.xlabel(r'$O_{NN}$', fontsize=14)
        plt.ylabel(r'$d\sigma/dO_{NN}$ [fb]', fontsize=14)
        y_min, y_max = plt.ylim()
        plt.ylim(y_min, y_max + 0.20 * (y_max - y_min))
        plt.xticks(fontsize=12)  
        plt.yticks(fontsize=12)
        plt.legend(frameon=False, fontsize=12)
        plt.savefig(f"O_NN_{interation}_{region}_jj.png", dpi=1000, bbox_inches="tight")
        plt.show()

        fractional_uncertainty_0 = np.divide(
            uncertainty_0, differential_cross_section_0,
            out=np.zeros_like(uncertainty_0),
            where=differential_cross_section_0 != 0
        )

        print("O_NN Fractional Uncertainty:", fractional_uncertainty_0)

In [6]:
def __main__():
    vbs_data = DataLoader("VBS_data_cwtil_2.feather", ['Electron.Eta', 'Electron.Phi', 'Muon.Eta', 'Muon.Phi', 'Jet.Eta', 'Jet.Phi'])
    model_filename = "vbs_lj_eta_phi.keras"

    param_grid = {
            'learning_rate': [0.001, 0.01],
            'neurons': [16, 32, 64],
            'layers': [2, 4],
            'reg_strength': [0, 0.0001, 0.001],
            'patience': [2],
            'batch_size': [128],
            'dropout_rate': [0, 0.2],
        }

    if os.path.exists(model_filename):
        print(f"Loading existing model for {model_filename}")
    else:
        print("No existing model found, training new model")
        vbs_model = MLModel()
        vbs_model.grid_search(vbs_data.x_data, vbs_data.y_data, param_grid, model_filename)
    
    vbs_model, vbs_history = MLModel.load_ML_model(model_filename)

    vbs_results = vbs_model.predict(vbs_data.x_data)
    Plotter.plot_ML_observable(vbs_results, vbs_data.weight_data, "VBS", 2)
    Plotter.plot_loss(vbs_history)
    Plotter.plot_accuracy(vbs_history)

if __name__ == '__main__':
    __main__()

x_data, y_data, weight_data (169076, 72) (169076, 1) (169076, 1)
No existing model found, training new model
Fitting 2 folds for each of 72 candidates, totalling 144 fits

Running fit 1/144

Running fit 1/144

Running fit 1/144

Running fit 1/144

Running fit 1/144

Running fit 1/144

Running fit 1/144

Running fit 1/144


2025-03-04 17:09:36.144942: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19477440 exceeds 10% of free system memory.
2025-03-04 17:09:36.144957: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19477440 exceeds 10% of free system memory.
2025-03-04 17:09:36.144981: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19477440 exceeds 10% of free system memory.
2025-03-04 17:09:36.144994: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19477440 exceeds 10% of free system memory.
2025-03-04 17:09:36.144993: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19477440 exceeds 10% of free system memory.
2025-03-04 17:09:36.145010: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19477440 exceeds 10% of free system memory.
2025-03-04 17:09:36.145096: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] A

Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50


2025-03-04 17:09:36.373876: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19477440 exceeds 10% of free system memory.


529/529 - 6s - 11ms/step - accuracy: 0.5010 - loss: 0.7025 - val_accuracy: 0.5020 - val_loss: 0.6955
Epoch 2/50
529/529 - 6s - 11ms/step - accuracy: 0.5043 - loss: 0.7185 - val_accuracy: 0.5005 - val_loss: 0.7089
529/529 - 6s - 11ms/step - accuracy: 0.5052 - loss: 0.7010 - val_accuracy: 0.5049 - val_loss: 0.6960
Epoch 2/50
Epoch 2/50
529/529 - 6s - 11ms/step - accuracy: 0.4975 - loss: 0.7065 - val_accuracy: 0.5072 - val_loss: 0.6959
Epoch 2/50
529/529 - 6s - 11ms/step - accuracy: 0.5059 - loss: 0.7025 - val_accuracy: 0.5073 - val_loss: 0.6944
Epoch 2/50
529/529 - 6s - 11ms/step - accuracy: 0.5023 - loss: 0.7000 - val_accuracy: 0.5075 - val_loss: 0.6937
Epoch 2/50
529/529 - 6s - 11ms/step - accuracy: 0.4990 - loss: 0.7125 - val_accuracy: 0.5043 - val_loss: 0.7045
Epoch 2/50
529/529 - 6s - 11ms/step - accuracy: 0.4999 - loss: 0.7002 - val_accuracy: 0.5025 - val_loss: 0.6935
Epoch 2/50
529/529 - 2s - 4ms/step - accuracy: 0.5103 - loss: 0.6942 - val_accuracy: 0.5078 - val_loss: 0.6949
Epoc

...

2025-03-04 17:18:24.382218: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19477440 exceeds 10% of free system memory.


529/529 - 4s - 8ms/step - accuracy: 0.4990 - loss: 0.6936 - val_accuracy: 0.4989 - val_loss: 0.6933
Epoch 6/50
529/529 - 3s - 5ms/step - accuracy: 0.5793 - loss: 0.6687 - val_accuracy: 0.5894 - val_loss: 0.6626
Epoch 10/50
529/529 - 5s - 9ms/step - accuracy: 0.6534 - loss: 0.6217 - val_accuracy: 0.6494 - val_loss: 0.6229
Epoch 14/50
529/529 - 3s - 5ms/step - accuracy: 0.6274 - loss: 0.6375 - val_accuracy: 0.6434 - val_loss: 0.6322
Epoch 13/50
529/529 - 3s - 5ms/step - accuracy: 0.5051 - loss: 0.6930 - val_accuracy: 0.5143 - val_loss: 0.6893
Epoch 4/50
529/529 - 3s - 5ms/step - accuracy: 0.5845 - loss: 0.6646 - val_accuracy: 0.5964 - val_loss: 0.6587
Epoch 11/50
529/529 - 5s - 9ms/step - accuracy: 0.5714 - loss: 0.6639 - val_accuracy: 0.5724 - val_loss: 0.6616
Epoch 13/50
529/529 - 3s - 5ms/step - accuracy: 0.6457 - loss: 0.6281 - val_accuracy: 0.6439 - val_loss: 0.6290
Epoch 14/50
[CV] END batch_size=128, dropout_rate=0, layers=4, learning_rate=0.001, neurons=64, patience=2, reg_streng

2025-03-04 17:18:34.221052: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19477440 exceeds 10% of free system memory.
2025-03-04 17:18:34.244814: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19477440 exceeds 10% of free system memory.
2025-03-04 17:18:34.280987: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19477440 exceeds 10% of free system memory.


Epoch 1/50
Epoch 1/50

Running fit 1/144
Epoch 1/50


KeyboardInterrupt: 

In [8]:
def __main__():
    vbs_data = DataLoader("VBS_data_cwtil_2.feather", ['Electron.Eta', 'Electron.Phi', 'Muon.Eta', 'Muon.Phi'])
    model_filename = "vbs_l_eta_phi.keras"

    param_grid = {
            'learning_rate': [0.001, 0.005, 0.01],
            'neurons': [16, 32, 64],
            'layers': [2, 4],
            'reg_strength': [0, 0.0001, 0.001],
            'patience': [2],
            'batch_size': [128],
            'dropout_rate': [0],
        }

    if os.path.exists(model_filename):
        print(f"Loading existing model for {model_filename}")
    else:
        print("No existing model found, training new model")
        vbs_model = MLModel()
        vbs_model.grid_search(vbs_data.x_data, vbs_data.y_data, param_grid, model_filename)
    
    vbs_model, vbs_history = MLModel.load_ML_model(model_filename)

    vbs_results = vbs_model.predict(vbs_data.x_data)
    Plotter.plot_ML_observable(vbs_results, vbs_data.weight_data, "VBS", 2)
    Plotter.plot_loss(vbs_history)
    Plotter.plot_accuracy(vbs_history)

if __name__ == '__main__':
    __main__()

Data shapes:  (84538, 48) (84538, 1) (84538, 1)
No existing model found, training new model
Fitting 2 folds for each of 54 candidates, totalling 108 fits

Running fit 1/108

Running fit 1/108

Running fit 1/108

Running fit 1/108
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
265/265 - 3s - 12ms/step - accuracy: 0.5006 - loss: 0.6957 - val_accuracy: 0.5048 - val_loss: 0.6962
Epoch 2/50
265/265 - 3s - 12ms/step - accuracy: 0.5036 - loss: 0.6959 - val_accuracy: 0.5012 - val_loss: 0.6955
Epoch 2/50
265/265 - 3s - 12ms/step - accuracy: 0.5018 - loss: 0.6986 - val_accuracy: 0.5047 - val_loss: 0.6969
Epoch 2/50
265/265 - 3s - 12ms/step - accuracy: 0.4981 - loss: 0.7019 - val_accuracy: 0.5011 - val_loss: 0.6963
Epoch 2/50
265/265 - 1s - 4ms/step - accuracy: 0.5064 - loss: 0.6946 - val_accuracy: 0.4995 - val_loss: 0.6957
Epoch 3/50
265/265 - 1s - 4ms/step - accuracy: 0.5055 - loss: 0.6940 - val_accuracy: 0.5014 - val_loss: 0.6944
Epoch 3/50
265/265 - 1s - 4ms/step - accuracy: 0.5021 - loss: 0.696

KeyboardInterrupt: 